# Exploración de hiperparámetros

In [1]:
import pandas as pd
import numpy as np
pd.set_option("max_rows", None)
from os import listdir
from os.path import isfile, join
import re
import os
import sys
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
tab_c = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink',
         'tab:gray','tab:olive','tab:cyan']

from MCMC_steiner_tree import Annealing, pre_pros, plot_graph_by_edges, Trans, weight, Annealing, read_stp, check_steiner_tree

In [2]:
root = {
    "main": "",  # "/Proyecto Sim. Est.",
    "data":"I080/",
    "data_I320":"/I320/",
    "sols": "bnb/solutions/" 
}

## Importando resultados

In [3]:
f = open('gridsearch0/gridsearch.json')
grid_search = json.load(f)
print(grid_search)
f.close()

{'a': [0.25, 0.5, 0.75, 1.0], 'b': [0.0, 0.25, 0.5, 0.75, 1.0]}


In [4]:
f = open('gridsearch0/params.json')
params = json.load(f)
print(params)
f.close()

{'nf': 3000, 'save_rate': 5, 'repeat': 10}


In [5]:
stp_names = ['i080-001.stp', 'i080-011.stp', 'i080-021.stp', 'i080-031.stp', 'i080-041.stp', 'i080-101.stp',
             'i080-111.stp', 'i080-121.stp', 'i080-131.stp', 'i080-141.stp', 'i080-201.stp', 'i080-211.stp',
             'i080-221.stp', 'i080-231.stp', 'i080-241.stp', 'i080-301.stp', 'i080-311.stp', 'i080-321.stp',
             'i080-331.stp', 'i080-341.stp']

In [6]:
df_opt = pd.read_excel('Testset_I320.xlsx', sheet_name="I080")
df_opt = df_opt.set_index("Name")
df_opt.head()

,|V|,|E|,|T|,DC,time,Opt
Name,,,,,,
i080-001,80.0,120.0,6.0,P,s,1787.0
i080-002,80.0,120.0,6.0,P,s,1607.0
i080-003,80.0,120.0,6.0,P,s,1713.0
i080-004,80.0,120.0,6.0,P,s,1866.0
i080-005,80.0,120.0,6.0,P,s,1790.0


In [7]:
with open('gridsearch0/UNIFORMS.pickle', 'rb') as handle:
    uniforms = pickle.load(handle)

In [8]:
methods = ['DFS_node']
a_values = grid_search['a']
b_values = grid_search['b']
len_U = len(uniforms)

In [10]:
class grafo_resultado(object):
    def __init__(self,name,len_U,df=df_opt,folder=root['main']+root['data']):
        name = name if not '.stp' in name else name.replace('.stp','')
        df_edges_G, terminals = read_stp(folder+name+'.stp')
        _, _, dic_edges, _ = pre_pros(df_edges_G)
        self.id = name
        self.terminals = terminals
        self.dic_edges = dic_edges
        self.opt = df['Opt'][name]
        self.size = int(df['|V|'][name])
        self.length = int(df['|E|'][name])
        self.difficulty = df['DC'][name]
        self.time = df['time'][name]
        self._results_loaded = False
        self._weights_computed = False
        self.len_U = len_U
        self.density = 2*self.length/(self.size*(self.size-1))
    
    @staticmethod
    def load_cm_path(path):
        with open(path, 'rb') as handle:
            return pickle.load(handle)
        
    def get_results(self,methods,a_values=grid_search['a'],b_values=grid_search['b']):
        results = {}
        times = {}
        for m in methods:
            results[m] = {}
            times[m] = {}
            for i, a in enumerate(a_values):
                results[m][a] = {}
                times[m][a] = {}
                for j, b in enumerate(b_values):
                    results[m][a][b] = {}
                    times[m][a][b] = {}
                    for l in range(self.len_U):
                        file = 'gridsearch0/{}/{}_ixa={}_ixb={}_ixu={}_CM.pickle'.format(self.id,m,i,j,l)
                        results[m][a][b][l] = self.load_cm_path(file)
                        file = 'gridsearch0/{}/{}_ixa={}_ixb={}_ixu={}_times.pickle'.format(self.id,m,i,j,l)
                        times[m][a][b][l] = self.load_cm_path(file)
        self.results = results
        self.times = times
        self._results_loaded = True
    
    def compute_weight_cm(self):
        weight_cm = {}
        weight_avg, weight_std, weight_min, weight_max = {}, {}, {}, {}
        for m in methods:
            weight_cm[m] = {}
            weight_avg[m], weight_std[m] = {}, {}
            weight_min[m], weight_max[m] = {}, {}
            for a in a_values:
                weight_cm[m][a] = {}
                weight_avg[m][a], weight_std[m][a] = {}, {}
                weight_min[m][a], weight_max[m][a] = {}, {}
                for b in b_values:
                    weight_cm[m][a][b] = {}
                    weight_avg[m][a][b], weight_std[m][a][b] = {}, {}
                    weight_min[m][a][b], weight_max[m][a][b] = {}, {}
                    for l in range(self.len_U):
                        weight_cm[m][a][b][l] = [weight(x, self.dic_edges) for x in self.results[m][a][b][l]]
                    weight_avg[m][a][b], weight_std[m][a][b] = self.avg_weights(weight_cm[m][a][b])
                    weight_min[m][a][b], weight_max[m][a][b] = self.min_max_weights(weight_cm[m][a][b])
        self.weight_cm = weight_cm
        self.weight_avg = weight_avg
        self.weight_std = weight_std
        self.weight_min = weight_min
        self.weight_max = weight_max
        self._weights_computed = True
        self.compute_weight_final()
    
    @staticmethod
    def avg_weights(weights_unif):
        N = len(weights_unif)
        weights_unif = np.array(list(weights_unif.values()))
        weights_avg = np.mean(weights_unif,axis=0)
        weights_std = np.std(weights_unif,axis=0)
        return list(weights_avg), list(weights_std)
    
    @staticmethod
    def min_max_weights(weights_unif):
        N = len(weights_unif)
        weights_unif = np.array(list(weights_unif.values()))
        weights_min = np.min(weights_unif,axis=0)
        weights_max = np.max(weights_unif,axis=0)
        return list(weights_min), list(weights_max)
    
    def compute_weight_final(self):
        weight_final = {}
        weight_best = {}
        for m in methods:
            weight_final[m] = {}
            weight_best[m] = {}
            for a in a_values:
                weight_final[m][a] = {}
                weight_best[m][a] = {}
                for b in b_values:
                    weight_final[m][a][b] = {}
                    weight_best[m][a][b] = {}
                    for l in range(self.len_U):
                        if self._weights_computed:
                            weight_final[m][a][b][l] = self.weight_cm[m][a][b][l][-1]
                            weight_best[m][a][b][l] = np.min(self.weight_cm[m][a][b][l])
#                         else:
#                             x = self.results[m][a][b][l][-1]
#                             weight_final[m][a][b][l] = weight(x, self.dic_edges)
        self.weight_final = weight_final
        self.weight_best = weight_best

---

In [11]:
dic_grafos = {}
for grafo_id in stp_names:
    grafo = grafo_id[:-4]
    print(grafo)
    grafo_result = grafo_resultado(grafo,len_U)
    grafo_result.get_results(methods)
    grafo_result.compute_weight_cm()
    dic_grafos[grafo] = grafo_result

i080-001
i080-011
i080-021
i080-031
i080-041
i080-101
i080-111
i080-121
i080-131
i080-141
i080-201
i080-211
i080-221
i080-231
i080-241
i080-301
i080-311
i080-321
i080-331
i080-341


In [12]:
global_summarize = {}
for grafo_id in stp_names:
    grafo = grafo_id[:-4]
    print(grafo)
    grafo_result = dic_grafos[grafo]
    opt_stp = grafo_result.opt
    summarize_opt = {}
    max_len = 0
    for variant, d1 in grafo_result.weight_cm.items():
        for a, d2 in d1.items():
            for b, d3 in d2.items():
                for i in range(10):
                    d4 = d3[i]
                    last_state = d4[-1]
                    if last_state == opt_stp:
                        l = f"{variant}/{a}/{b}"
                        if l in summarize_opt.keys():
                            summarize_opt[l][0] += 1
                            summarize_opt[l][1].append(i)
                        else:
                            summarize_opt[l] = [1, [i]]
                l = f"{variant}/{a}/{b}"
                if l in summarize_opt.keys(): 
                    max_len = max(max_len, summarize_opt[l][0])
    global_summarize[grafo] = [max_len, summarize_opt]

i080-001
i080-011
i080-021
i080-031
i080-041
i080-101
i080-111
i080-121
i080-131
i080-141
i080-201
i080-211
i080-221
i080-231
i080-241
i080-301
i080-311
i080-321
i080-331
i080-341


In [13]:
global_cand_best_hyperparams = {}
for grafo_id in stp_names:
    grafo = grafo_id[:-4]
    GS_grafo = global_summarize[grafo]
    cand_best_hyperparams = []
    max_len = GS_grafo[0]
    for k, v in GS_grafo[1].items():
        if v[0] == max_len:
            cand_best_hyperparams.append((k, v[1]))
    
    global_cand_best_hyperparams[grafo] = [max_len, cand_best_hyperparams]

In [14]:
global_cand_best_hyperparams_iter_times = {}
for grafo_id in stp_names:
    grafo = grafo_id[:-4]
    grafo_result = dic_grafos[grafo]
    val_opt = grafo_result.opt
    info_array = []
    for params, uniforms in global_cand_best_hyperparams[grafo][1]:
        variant, a, b = params.split("/")
        a, b = float(a), float(b)
        mean_iter = 0
        mean_time = 0
        total_time = 0
        for uni in uniforms:
            _times = grafo_result.times[variant][a][b][uni]
            _cm = grafo_result.weight_cm[variant][a][b][uni]
            iter_first_opt = _cm.index(val_opt)
            time_first_opt = _times[iter_first_opt]
            mean_iter += iter_first_opt/len(uniforms)
            mean_time += time_first_opt/len(uniforms)
            total_time += _times[-1]/len(uniforms)
        info_array.append([params, (mean_iter, mean_time, total_time)])
    global_cand_best_hyperparams_iter_times[grafo] = [global_cand_best_hyperparams[grafo][0], info_array]

In [15]:
global_cand_best_hyperparams_iter_times['i080-001']

[8, [['DFS_node/0.75/1.0', (75.25, 0.6012730092502352, 4.924933514376335)]]]

In [16]:
global_best_hyperparams = {}
for grafo_id in stp_names:
    grafo = grafo_id[:-4]
    
    cands = global_cand_best_hyperparams_iter_times[grafo][1]
    if cands:
        global_best_hyperparams[grafo] =  [global_cand_best_hyperparams_iter_times[grafo][0], min(cands,  key=lambda x: x[1][1])]

In [17]:
global_best_hyperparams['i080-001']

[8, ['DFS_node/0.75/1.0', (75.25, 0.6012730092502352, 4.924933514376335)]]

In [18]:
col_grafo = []
col_variant = []
col_a = []
col_b = []
col_iter = []
col_time = []
col_unifs = []
col_total = []
for k, v in global_best_hyperparams.items():
    unifs, (params, (iter_, times_, total_)) = v
    col_grafo.append(k)
    variant, a, b = params.split("/")
    a, b = float(a), float(b)
    col_variant.append(variant)
    col_a.append(a)
    col_b.append(b)
    col_iter.append(iter_)
    col_time.append(times_)
    col_unifs.append(unifs)
    col_total.append(total_)
df_summarize_annealing = pd.DataFrame({
    "grafo": col_grafo,#.replace('i080-',''),
    #"variante": col_variant,
    "a": col_a,
    "b": col_b,
    "unifs (N°)": col_unifs,
    "iter (N°)": np.around(col_iter,decimals=3),
    "t. min (s)": np.around(col_time,decimals=3),
    "t. total (s)": np.around(col_total,decimals=3),
})
df_summarize_annealing

,grafo,a,b,unifs (N°),iter (N°),t. min (s),t. total (s)
0,i080-001,0.75,1.00,8,75.25,0.601,4.925
1,i080-021,1.00,0.25,2,253.50,7.007,20.550
2,i080-031,0.50,0.25,10,48.60,0.393,4.534
3,i080-041,1.00,0.75,1,40.00,0.486,10.232
4,i080-101,0.50,0.25,10,111.90,1.570,8.672
5,i080-111,0.75,0.50,1,30.00,0.331,5.773
6,i080-121,0.25,0.50,1,215.00,6.508,21.243
7,i080-131,0.75,0.50,1,15.00,0.156,6.720
8,i080-141,0.25,0.50,1,136.00,2.543,12.072
9,i080-201,1.00,0.50,10,116.30,2.007,12.643


In [19]:
files = [g for g in stp_names if isfile(root["sols"]+f"sol_{g[:-4]}.txt")]
col_grafo_PLM = []
col_time_PLM = []
for grafo_id in files:
    grafo = grafo_id[:-4]
    with open(root["sols"]+f'sol_{grafo}.txt') as f:
        lines = f.readlines()
    time_ = float(lines[0].split()[1])
    value_ = float(lines[1].split()[1])
    opt_grafo = dic_grafos[grafo].opt
    if abs(opt_grafo - value_) < 1:
        col_grafo_PLM.append(grafo)
        col_time_PLM.append(time_)  
df_summarize_PLM = pd.DataFrame({
    "grafo": col_grafo_PLM,
    "t. PLM (s)": col_time_PLM,
})      

In [20]:
df_summarize = df_summarize_annealing.join(df_summarize_PLM.set_index('grafo'), on='grafo')
df_summarize

,grafo,a,b,unifs (N°),iter (N°),t. min (s),t. total (s),t. PLM (s)
0,i080-001,0.75,1.00,8,75.25,0.601,4.925,0.725
1,i080-021,1.00,0.25,2,253.50,7.007,20.550,6.659
2,i080-031,0.50,0.25,10,48.60,0.393,4.534,0.167
3,i080-041,1.00,0.75,1,40.00,0.486,10.232,1.239
4,i080-101,0.50,0.25,10,111.90,1.570,8.672,0.191
5,i080-111,0.75,0.50,1,30.00,0.331,5.773,16.074
6,i080-121,0.25,0.50,1,215.00,6.508,21.243,199.644
7,i080-131,0.75,0.50,1,15.00,0.156,6.720,0.352
8,i080-141,0.25,0.50,1,136.00,2.543,12.072,38.082
9,i080-201,1.00,0.50,10,116.30,2.007,12.643,2.713


In [21]:
col_grafo_info = []
col_density = []
col_terminals = []
col_size = []
col_length = []
col_difficulty = []
col_time_str = []

for k, v in dic_grafos.items():
    col_grafo_info.append(k)
    col_density.append(v.density)
    col_terminals.append(len(v.terminals))
    col_size.append(v.size)
    col_length.append(v.length)
    col_difficulty.append(v.difficulty)
    col_time_str.append(v.time)

df_info = pd.DataFrame({
    "grafo": col_grafo_info,
    "densidad": np.around(col_density,decimals=3),
    "$|$T$|$": col_terminals,
    "$|$V$|$": col_size,
    "$|$E$|$": col_length,
})     
df_info

,grafo,densidad,$|$T$|$,$|$V$|$,$|$E$|$
0,i080-001,0.038,6,80,120
1,i080-011,0.111,6,80,350
2,i080-021,1.000,6,80,3160
3,i080-031,0.051,6,80,160
4,i080-041,0.200,6,80,632
5,i080-101,0.038,8,80,120
6,i080-111,0.111,8,80,350
7,i080-121,1.000,8,80,3160
8,i080-131,0.051,8,80,160
9,i080-141,0.200,8,80,632


In [22]:
df_summarize.join(df_info.set_index('grafo'), on='grafo')["grafo  $|$V$|$  $|$E$|$  $|$T$|$  densidad  a  b  unifs (N°)  iter (N°)  t. min (s)  t. total (s)  t. PLM (s)".split("  ")]

,grafo,$|$V$|$,$|$E$|$,$|$T$|$,densidad,a,b,unifs (N°),iter (N°),t. min (s),t. total (s),t. PLM (s)
0,i080-001,80,120,6,0.038,0.75,1.00,8,75.25,0.601,4.925,0.725
1,i080-021,80,3160,6,1.000,1.00,0.25,2,253.50,7.007,20.550,6.659
2,i080-031,80,160,6,0.051,0.50,0.25,10,48.60,0.393,4.534,0.167
3,i080-041,80,632,6,0.200,1.00,0.75,1,40.00,0.486,10.232,1.239
4,i080-101,80,120,8,0.038,0.50,0.25,10,111.90,1.570,8.672,0.191
5,i080-111,80,350,8,0.111,0.75,0.50,1,30.00,0.331,5.773,16.074
6,i080-121,80,3160,8,1.000,0.25,0.50,1,215.00,6.508,21.243,199.644
7,i080-131,80,160,8,0.051,0.75,0.50,1,15.00,0.156,6.720,0.352
8,i080-141,80,632,8,0.200,0.25,0.50,1,136.00,2.543,12.072,38.082
9,i080-201,80,120,16,0.038,1.00,0.50,10,116.30,2.007,12.643,2.713
